In [49]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 40
POSITION = "lying_down_left"
MODEL_NAME = f"position:{POSITION}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"


### This code takes in a position, eg "sitting_or_standing", "lying_down_left" etc and trains a model for just classifying activities of that position

In [50]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split


POSITIONS = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

STANDING_OR_SITTING_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",
    
    "sitting&talking",
    "sitting&eating",
    "sitting&singing",
    "sitting&laughing",
    
    "standing&talking",
    "standing&eating",
    "standing&singing",
    "standing&laughing",

}

STANDING_OR_SITTING_ACTIVITIES_COMBINED = {
    "sitting_or_standing&coughing",
    "sitting_or_standing&hyperventilating",
    "sitting_or_standing&normal_breathing",
    
    
    "sitting_or_standing&talking",
    "sitting_or_standing&eating",
    "sitting_or_standing&singing",
    "sitting_or_standing&laughing",
    


}


LYING_DOWN_LEFT_ACTIVITIES = {
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    "lying_down_left&normal_breathing"

}

LYING_DOWN_RIGHT_ACTIVITIES = {
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing",
    "lying_down_right&normal_breathing"
}

LYING_DOWN_BACK_ACTIVITIES = {
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
    "lying_down_back&normal_breathing"
}

LYING_DOWN_STOMACH_ACTIVITIES = {
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing",
    "lying_down_stomach&normal_breathing"
}

DATA_DIRECTORY = "./all_respeck"

position_activities = {"sitting_or_standing": STANDING_OR_SITTING_ACTIVITIES,
                       "lying_down_left": LYING_DOWN_LEFT_ACTIVITIES,
                       "lying_down_right": LYING_DOWN_RIGHT_ACTIVITIES,
                       "lying_down_back": LYING_DOWN_BACK_ACTIVITIES,
                       "lying_down_stomach": LYING_DOWN_STOMACH_ACTIVITIES}

POSSIBLE_ACTIVITIES = position_activities[POSITION]


In [51]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size))
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)))
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [52]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in POSSIBLE_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                position = key.split("&")[0]
                activity = key.split("&")[1]
                if position == "standing" or position == "sitting":
                    tagged_data.append(("sitting_or_standing" + "&" + activity, sequence))
                else:
                    tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [53]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        #layers.Conv1D(256, 3, activation='relu'),
        #layers.MaxPooling1D(2),
        #layers.Conv1D(512, 3, activation='relu'),
        #layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        #layers.Dense(256, activation='relu'),
        #layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [54]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Combine all sequences and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]

if POSITION == "sitting_or_standing":
    possible_activities = STANDING_OR_SITTING_ACTIVITIES_COMBINED
else:
    possible_activities = POSSIBLE_ACTIVITIES

# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(possible_activities)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, possible_activities, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/models_for_presentation/lying_down_left_model.keras")

there are 7439 tagged sequences in the dataset
Epoch 1/40
596/596 [==============================] - 2s 3ms/step - loss: 1.7613 - accuracy: 0.2151 - val_loss: 1.7227 - val_accuracy: 0.2419
Epoch 2/40
596/596 [==============================] - 2s 3ms/step - loss: 1.4504 - accuracy: 0.3556 - val_loss: 1.2952 - val_accuracy: 0.4099
Epoch 3/40
596/596 [==============================] - 2s 3ms/step - loss: 1.2867 - accuracy: 0.4095 - val_loss: 1.2848 - val_accuracy: 0.4019
Epoch 4/40
596/596 [==============================] - 2s 3ms/step - loss: 1.2258 - accuracy: 0.4534 - val_loss: 1.1929 - val_accuracy: 0.4704
Epoch 5/40
596/596 [==============================] - 2s 3ms/step - loss: 1.1424 - accuracy: 0.4900 - val_loss: 1.1126 - val_accuracy: 0.4825
Epoch 6/40
596/596 [==============================] - 2s 3ms/step - loss: 1.0825 - accuracy: 0.5176 - val_loss: 1.0434 - val_accuracy: 0.5618
Epoch 7/40
596/596 [==============================] - 2s 3ms/step - loss: 1.0070 - accuracy: 0.5596 -